<a href="https://colab.research.google.com/github/amyth18/CS598-Deep-Learning-Final-Project/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install gensim --upgrade
! pip install psutil
! pip install transformers

     |████████████████████████████████| 24.1 MB 1.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 4.0 MB 14.6 MB/s 
     |████████████████████████████████| 6.6 MB 97.0 MB/s 
     |████████████████████████████████| 895 kB 90.1 MB/s 
     |████████████████████████████████| 596 kB 89.4 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import torch

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Load Data

In [4]:
# read data
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

Negative filters (e.g. "no sign of breath problem").

Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

Generate TF-IDF vector for each symptom entity.

Generate multi-hot encoding for y


## Utility Routines For Data Processing

In [5]:
from typing import List
import re
import nltk
from transformers import AutoTokenizer, pipeline,  AutoModelForTokenClassification

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
        tokenizer = AutoTokenizer.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        model = AutoModelForTokenClassification.from_pretrained(
            "samrawal/bert-base-uncased_clinical-ner")
        symptom_extractor = pipeline('ner', model=model, tokenizer=tokenizer,
                                     device=0)
        text_anno = symptom_extractor(self.dframe["TEXT"].tolist(), 
                                           batch_size=256)
        self.data = list(zip(text_anno, self.dframe["TEXT"].tolist()))
        


    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    def extract_symptoms(self, input_anno_text) -> List[str]:
      """
      The method extract the symptom phrases from the input
      """
      extractions, text = input_anno_text
      span_extract = []
      for extract in extractions:
        if 'problem' in extract['entity']:
          span_extract.append((extract['start'], extract['end']))

      # Check if this span_extract is empty
      if not span_extract:
        return []
      
      span_st = span_extract[0][0]
      final_span = []
      final_end = span_extract[0][1]
      for idx, (st, end) in enumerate(span_extract):
        if idx == 0:
          continue
        if st - span_extract[idx-1][1] <= 2:
          final_end = end
          if idx == len(span_extract) - 1:
            final_span.append((span_st, final_end))
        else:
          final_span.append((span_st, final_end))
          span_st = st
          final_end = end

      text_extracts = [text[st:end].replace("\n", " ") for st, end in final_span]
      return text_extracts

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          symptoms = " ".join(self.extract_symptoms(self.data[idx]))
          text = self.sanitize_text(symptoms)
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
def pad_dataset(dataset, vec_size):
  seq_lengths = list()

  for idx in range(len(dataset)):
    seq_lengths.append(len(dataset[idx]))
  max_seq_length = max(seq_lengths)

  padded_dataset = torch.zeros([len(dataset), max_seq_length, vec_size], 
                               dtype=torch.float)
  for i in range(len(dataset)):
    for j in range(len(dataset[i])):
      padded_dataset[i][j] = torch.FloatTensor(dataset[i][j])
  
  return padded_dataset

In [7]:
from datetime import datetime
import pytz

def get_model_file_name():
  return "/content/drive/My Drive/DLH Final Project/models/model-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

def get_stats_file_name():
  return "/content/drive/My Drive/DLH Final Project/stats/stats-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

def get_results_file_name():
  return "/content/drive/My Drive/DLH Final Project/stats/results-" + \
                  datetime.now(pytz.timezone('Asia/Kolkata')).strftime(
                      "%d-%m-%Y-%H-%M-%S")

## Data Filtering and Tranformation

In [8]:
df_icd_codes = pd.read_csv(
    "/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

Get top #50 ICD9 codes 

In [9]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()

Filter data to include admimission with top 50 diseases only and group and reorganize data in the following format <subject_id, hadm_id, [icd_code1, icd_code2 ...]>

In [10]:
df_admissions_with_top_diseases = \
df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]

df_admissions_with_top_diseases = \
df_admissions_with_top_diseases.groupby(
['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].apply(
        list).to_frame().reset_index()

Now select discharge summaries for the admimissions that contain atleast one of the top #50 ICD codes.

In [11]:
df_dataset = pd.merge(df, df_admissions_with_top_diseases, 
                       on=["SUBJECT_ID", "HADM_ID"])

df_dataset = df_dataset[df_dataset["CATEGORY"] 
                          == 'Discharge summary'].reset_index()
# free up some memory
# del df

In [12]:
df_dataset.head(3)

,index,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,ICD9_CODE
0,0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,"[42731, 2762, 5070, 5119]"
1,1,170,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,"[42731, 2762, 5070, 5119]"
2,32,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,"[51881, 486, 2761, 2449, 311]"


Need to covert the ICD9 column to multi-hot encoding, we keep the old column with list of codes and added and new column with multi-hot encoding representation.

In [13]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

In [14]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_dataset)):
  icd_codes = df_dataset.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

# new add a new column with multi-hot encoding.
df_dataset['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

Extract symptoms from the text. Note: currently we treat all tokens as symptoms need to add all the filters discussed in the paper later. So we added a column called "SYMPTOMS" which is simply tokenized form of "TEXT" after basic sanitization.

In [15]:
from tqdm import tqdm
sgen = MySentences(df_dataset)
symptom_col = list()
for s in tqdm(sgen):
  symptom_col.append(s)

# add the new column to the dataset.
df_dataset["SYMPTOMS"] = symptom_col

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

55988it [00:10, 5270.14it/s]


In [16]:
df_dataset = df_dataset[df_dataset.apply(lambda x: len(x.SYMPTOMS) > 0, axis=1)]
df_dataset = df_dataset.reset_index()
df_dataset.to_csv("/content/drive/My Drive/DLH Final Project/kbr_df_dataset.csv")

# df_dataset = pd.read_csv("/content/drive/My Drive/DLH Final Project/kbr_df_dataset.csv")
del sgen

## Generate Word2Vec Embeddings

Word2Vec training using gensim.

In [17]:
# NOTE: commenting this part so that we dont run this by mistake.
# we only need to perform this once at the start of the project.

# import gensim
# sgen = MySentences(df_dataset) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model")

## Construct dataset with Word2Vec embeddings

In [18]:
from gensim.models import Word2Vec
model = Word2Vec.load('/content/drive/My Drive/DLH Final Project/mimic3/word2vec-4.model')

In [19]:
X_word2vec = list()
for idx in range(len(df_dataset)):
  # ignore words in not vocabulary
  symptoms = df_dataset["SYMPTOMS"][idx]
  symptoms_emb = [model.wv[s] for s in symptoms if s in model.wv]
  X_word2vec.append(symptoms_emb)

# pad the dataset.
# X_word2vec = pad_dataset(X_word2vec)

In [20]:
# import pickle
# pfile = open("/content/drive/My Drive/DLH Final Project/X_word2vec", "ab")
# pickle.dump(X_word2vec, pfile)
# pfile.close()

# Construct data with TF-IDF encoding

In [21]:
import numpy as np
import itertools

vocab_size = len(model.wv)
tf = np.zeros((len(model.wv), len(top_icd_codes)))


for idx in range(len(df_dataset)):
  # XXX: TODO currently we treat all tokens from "TEXT" as sypmtoms
  # get the icd codes for this discharge summary
  symptoms = df_dataset['SYMPTOMS'][idx]
  icd_codes = df_dataset['ICD9_CODE'][idx]
  # create a cross product of symptoms and icd codes
  # and update tf matrix. tf matrix keeps count of how many 
  # (i.e frequency) times <symptom, icd code> pair occur in our dataset.
  for pair in itertools.product(symptoms, icd_codes):
    # update count of each (symptom, icd_code) pair to compute TF
    if pair[0] in model.wv:
      tf[model.wv.get_index(pair[0])][icd_code_to_idx[pair[1]]] += 1

# Complete the TF-IDF matrix computation.
# Compute the number of ICD Codes (i.e diseaes) each 
# symptom is associated with.
D_i = np.sum(tf > 0, axis=1)
print(D_i.shape)

log_N_Di = np.log(len(top_icd_codes)/D_i)
tf_idf = (tf.T * log_N_Di).T
print(tf_idf.shape)

(64259,)
(64259, 50)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


In [22]:
# build the X_tfidf dataset
X_tf_idf = list()
for idx in range(len(df_dataset)):
  symptoms = df_dataset["SYMPTOMS"][idx]
  # get tf-idf vector for each symptom
  # ignore words in not vocabulary
  symptoms_tf_idf = [tf_idf[model.wv.get_index(s)] \
                     for s in symptoms if s in model.wv]
  X_tf_idf.append(symptoms_tf_idf)

# pad the dataset.
# X_tf_idf = pad_dataset(X_tf_idf)

In [23]:
# import pickle
# pfile = open("/content/drive/My Drive/DLH Final Project/X_tf_idf", "ab")
# pickle.dump(tf_idf, pfile)
# pfile.close()

# Construct Y (Multihot Encoding)

In [24]:
# multi-hot encoding for ICD codes diagnosed.
y = df_dataset['ICD9_CODE_ENCODED'].to_list()

In [25]:
print(len(X_word2vec))
print(len((X_tf_idf)))
print(len(y))

54939
54939
54939


# Define Dataset and DataLoaders

In [48]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

def collate_fn(data):
  x_w2v, x_tidf, y_batch = zip(*data)
  x_w2v = pad_dataset(x_w2v, 100)
  x_tidf = pad_dataset(x_tidf, 50)
  y_batch = torch.FloatTensor(y_batch)
  x_w2v = x_w2v.cuda() if torch.cuda.is_available() else x_w2v
  x_tidf = x_tidf.cuda() if torch.cuda.is_available() else x_tidf
  y_batch = y_batch.cuda() if torch.cuda.is_available() else y_batch

  return x_w2v, x_tidf, y_batch

class CustomDataset(Dataset):

  def __init__(self, X_w2v, X_tfidf, y):              
    self.X_w2v = X_w2v
    self.X_tfidf = X_tfidf
    self.y = y
    
  def __len__(self):                
    return len(self.y)
    
  def __getitem__(self, index):          
    # your code here
    return self.X_w2v[index], self.X_tfidf[index], self.y[index]

dataset = CustomDataset(X_word2vec, X_tf_idf, y)

split = int(len(dataset)*0.8)
lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=400, 
                          collate_fn=collate_fn)

test_loader = DataLoader(test_dataset, shuffle=True, batch_size=400, 
                         collate_fn=collate_fn)

# Model Definition

TODO: drop out rate?

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM(nn.Module):
  def __init__(self, input_dim, embedding_dim, output_dim):   
    super(BiLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size=input_dim, 
                        hidden_size=embedding_dim,
                        num_layers=1,
                        bidirectional=True,
                        batch_first=True)
    
    self.linear = nn.Linear(embedding_dim*2, 
                            output_dim)
  
  def forward(self, X):
    out, (hn, cn) = self.lstm(X)    
    emb = torch.mean(out, dim=1)
    output = torch.sigmoid(self.linear(emb))
    return output

In [35]:
class DiseasePredictionModel(nn.Module):
  def __init__(self, weight=0.4):    
    super(DiseasePredictionModel, self).__init__()
    self.weight = 0.4    
    self.w2v_lstm = BiLSTM(input_dim=100, embedding_dim=50, output_dim=50)
    self.tf_idf_lstm = BiLSTM(input_dim=50, embedding_dim=50, output_dim=50)
  
  def forward(self, X_w2v, X_tidf):
    pred1 = self.w2v_lstm(X_w2v)
    pred2 = self.tf_idf_lstm(X_tidf)
    # compute the weighted average of predictions
    # from the 2 models.
    return self.weight * pred1 + (1-self.weight) * pred2

# Model Training

In [36]:
model = DiseasePredictionModel()
if torch.cuda.is_available():
  model.cuda()

loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Number of trainable parameters

In [30]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

111700

In [31]:
import psutil
print(psutil.Process().memory_info())
print(psutil.virtual_memory())
print(torch.cuda.memory_allocated())

pmem(rss=11116875776, vms=36147789824, shared=1870974976, text=4620288, lib=0, data=10203312128, dirty=0)
svmem(total=54767017984, available=46653997056, percent=14.8, used=10601730048, free=21669404672, active=7663321088, inactive=24512430080, buffers=151396352, cached=22344486912, shared=17391616, slab=591757312)
448512


In [49]:
import psutil
import time
import pickle

main_memory_usage = list()
gpu_memory_usage = list()
gpu_time = list()
train_loss = list()

for e in range(100):
  model.train()
  epoc_train_loss = 0
  main_memory_before = psutil.virtual_memory().used
  gpu_memory_before = torch.cuda.memory_allocated()
  start_time = time.time()

  # iterate over data in mini batches.
  for x_w2v, x_tidf, y_batch in train_loader:    
    model.zero_grad()
    pred = model(x_w2v, x_tidf)
    l = loss(pred, y_batch)
    l.backward()
    optimizer.step()    
    epoc_train_loss += l.item()
    
  # print epoc level training loss.
  print(f"epoc: {e}: Train Loss: {epoc_train_loss/len(train_loader)}")
  
  # collect cpu and memory stats.
  memory_used = psutil.virtual_memory().used
  gpu_memory_used = torch.cuda.memory_allocated()
  run_time = time.time() - start_time
  print(f"time: {run_time} memory_used: {memory_used} gpu_memory_used: {gpu_memory_used}")
  print("\n")

  train_loss.append(epoc_train_loss/len(train_loader))
  main_memory_usage.append(memory_used)
  gpu_memory_usage.append(gpu_memory_used)
  gpu_time.append(run_time)
  # end of one epoc

# save the model
torch.save(model.state_dict(), get_model_file_name())
# print and collect stats.
print(psutil.virtual_memory())

stats = {
    "gpu_mem": gpu_memory_usage,
    "main_mem": main_memory_usage,
    "gpu_time": gpu_time,
    "vmm_info": psutil.virtual_memory()
}

with open(get_stats_file_name(), "ab") as sfile:
  pickle.dump(stats, sfile)

epoc: 0: Train Loss: 0.22699457813392987
time: 35.01112174987793 memory_used: 11161112576 gpu_memory_used: 21208576


epoc: 1: Train Loss: 0.22569478519938208
time: 35.60397148132324 memory_used: 11161698304 gpu_memory_used: 43171328


epoc: 2: Train Loss: 0.2245036471973766
time: 35.16093587875366 memory_used: 11160223744 gpu_memory_used: 44648960


epoc: 3: Train Loss: 0.22345253432338888
time: 35.07908749580383 memory_used: 11161178112 gpu_memory_used: 43369984


epoc: 4: Train Loss: 0.22242891693657094
time: 35.129961252212524 memory_used: 11159425024 gpu_memory_used: 48637440


epoc: 5: Train Loss: 0.22147019695151937
time: 35.20118069648743 memory_used: 11159781376 gpu_memory_used: 44648960


epoc: 6: Train Loss: 0.2204536740075458
time: 35.10628914833069 memory_used: 11159379968 gpu_memory_used: 45517824


epoc: 7: Train Loss: 0.2197259632023898
time: 35.49019718170166 memory_used: 11159326720 gpu_memory_used: 40649728


epoc: 8: Train Loss: 0.21875532106919723
time: 35.22235226

# Model Evaluation

In [51]:
# model = DiseasePredictionModel()
# model.load_state_dict(
#    torch.load("/content/drive/My Drive/DLH Final Project/model"))

from sklearn.metrics import precision_recall_fscore_support

model.eval()
y_pred_all = list()
y_true_all = list()

for x_w2v, x_tidf, y_batch in test_loader:
  y_pred = model(x_w2v, x_tidf)
  y_pred = y_pred > 0.2
  y_pred_all.extend(y_pred.detach().to('cpu').numpy())
  y_true_all.extend(y_batch.detach().to('cpu').numpy())

p1, r1, f1, s1 = precision_recall_fscore_support(y_true_all, y_pred_all, 
                                                 average="micro")
print(f"Micro Averaging. Precision: {p1}, Recall: {r1}, F1 Score: {f1}")

p2, r2, f2, s2 = precision_recall_fscore_support(y_true_all, y_pred_all, 
                                                 average="macro")

print(f"Macro Averaging. Precision: {p2}, Recall: {r2}, F1 Score: {f2}")

results = {
    "micro": [p1, r1, f1],
    "macro": [p2, r2, f2]
}

with open(get_results_file_name(), "ab") as rfile:
  pickle.dump(results, rfile)

Micro Averaging. Precision: 0.40808268051446783, Recall: 0.6468433657348709, F1 Score: 0.5004437524328533
Macro Averaging. Precision: 0.37372860900694455, Recall: 0.5587988247514366, F1 Score: 0.44099536878270906
